# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 2 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/NotAnnotated/Regular/AveragePooled/split_features_1s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[-0.01720456, -0.9333485 , -0.9391832 , ..., -0.82813597,
         -0.8620985 , -0.9794059 ],
        [-0.01720306, -0.9333132 , -0.93914014, ..., -0.8327555 ,
         -0.86637986, -0.97935015],
        [-0.01720168, -0.9332716 , -0.93908674, ..., -0.847576  ,
         -0.8784226 , -0.9896085 ],
        ...,
        [-0.01720948, -0.55143994, -0.52205884, ..., -0.87508094,
         -0.90587527, -0.9997576 ],
        [-0.0171332 , -0.55125844, -0.51631904, ..., -0.8850505 ,
         -0.9144034 , -0.99985427],
        [-0.01714398, -0.5486961 , -0.5132266 , ..., -0.88104194,
         -0.911118  , -1.        ]], dtype=float32),
 'mfcc': array([[-1.        , -0.02898505, -0.10768069, ...,  0.01827751,
          0.02722204,  0.04308926],
        [-1.        , -0.02902972, -0.1065027 , ...,  0.01701666,
          0.02487057,  0.04410085],
        [-1.        , -0.0125336 , -0.09841443, ...,  0.02036034,
          0.02327715,  0.04241924],
        ...,
        [-0.9

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(42578,)

array([ 7, 16, 19,  0,  1,  7,  6,  3,  1,  1, 19,  7,  9,  7, 12],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(42578, 128)

array([-0.02185728, -0.3939447 , -0.41050395, -0.50029534, -0.56085765,
       -0.6310321 , -0.6548134 , -0.6388136 , -0.63982546, -0.6337072 ,
       -0.64285135, -0.65793073, -0.6480776 , -0.6594148 , -0.6366135 ,
       -0.654948  , -0.6432404 , -0.6395858 , -0.6798748 , -0.6723758 ,
       -0.66720843, -0.69844174, -0.7145976 , -0.7270461 , -0.7289734 ,
       -0.75110966, -0.7298497 , -0.696753  , -0.68113   , -0.683587  ,
       -0.6695822 , -0.66871876, -0.66008353, -0.6631871 , -0.65656364,
       -0.6460304 , -0.6575156 , -0.6447011 , -0.6472018 , -0.6569579 ,
       -0.66957086, -0.67637694, -0.67121893, -0.6779927 , -0.68991876,
       -0.6964    , -0.7139648 , -0.7335337 , -0.7551437 , -0.7620292 ,
       -0.76579344, -0.7464363 , -0.71748465, -0.6991841 , -0.6900108 ,
       -0.6999048 , -0.70721406, -0.7059485 , -0.6748798 , -0.66050416,
       -0.6367882 , -0.62208366, -0.6234393 , -0.6094999 , -0.595338  ,
       -0.602499  , -0.62069196, -0.66656387, -0.7080173 , -0.73

'mfcc'

(42578, 20)

...

(42578, 84)

array([-12.80637 , -10.178921, -15.586308, -16.283943, -17.871552,
       -15.207264, -15.745362, -16.976467, -19.377678, -20.634443,
       -19.823278, -16.28474 , -11.230347, -13.031438, -22.283443,
       -23.941479, -24.802055, -24.640802, -26.405006, -22.172436,
       -23.847929, -29.4433  , -32.76458 , -32.2666  , -29.44668 ,
       -31.624424, -37.031097, -38.127777, -39.327816, -38.604076,
       -38.72474 , -38.8844  , -37.452694, -36.85629 , -38.343513,
       -37.406395, -37.946125, -37.184864, -38.44911 , -39.839275,
       -38.192352, -39.987495, -38.41094 , -37.54097 , -38.557415,
       -38.12515 , -37.687935, -40.436245, -40.11137 , -41.43069 ,
       -43.12546 , -45.00774 , -46.521675, -45.079754, -42.41699 ,
       -41.021828, -39.632954, -39.205273, -38.933006, -37.991035,
       -38.70373 , -40.62161 , -40.877434, -43.137257, -46.641926,
       -46.76184 , -43.449017, -42.309822, -41.863094, -38.54685 ,
       -35.893127, -34.787994, -36.57433 , -41.203064, -46.978

'id'

(42578,)

'XC184429.mp3'

In [13]:
display(val_y.shape)
display(val_y[:15])

(13083,)

array([ 3,  5, 11, 11, 18, 12, 19, 18,  1,  2,  1,  4, 17,  0, 11],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(13083, 128)

array([-0.01719964, -0.9332614 , -0.9346287 , -0.92839473, -0.9182133 ,
       -0.91571504, -0.90023047, -0.9096828 , -0.9013315 , -0.8832822 ,
       -0.8897478 , -0.8778112 , -0.8611843 , -0.87501085, -0.86042863,
       -0.87427056, -0.8929696 , -0.8929555 , -0.89963925, -0.90093535,
       -0.90265375, -0.90664506, -0.9189995 , -0.91942257, -0.9140625 ,
       -0.9264467 , -0.93026924, -0.9284385 , -0.934406  , -0.92323446,
       -0.93527853, -0.9328583 , -0.9292407 , -0.9284236 , -0.9269353 ,
       -0.9171119 , -0.9047776 , -0.8865346 , -0.90121865, -0.90096414,
       -0.9121269 , -0.91248345, -0.893796  , -0.90334725, -0.914868  ,
       -0.9018331 , -0.8862433 , -0.8832799 , -0.8790057 , -0.87130845,
       -0.86195666, -0.8688272 , -0.8709505 , -0.85542107, -0.8491089 ,
       -0.8495483 , -0.8412429 , -0.84774375, -0.84960586, -0.85435015,
       -0.85175586, -0.85534483, -0.8512846 , -0.85755765, -0.8516211 ,
       -0.8539241 , -0.8570129 , -0.83969903, -0.8464547 , -0.84

'mfcc'

(13083, 20)

...

(13083, 84)

array([-35.882477, -38.292088, -37.4063  , -40.330284, -42.694447,
       -42.67396 , -44.58036 , -46.30381 , -47.487316, -47.476955,
       -48.25535 , -50.481647, -55.270306, -54.9184  , -54.514065,
       -53.466293, -53.302032, -54.178078, -56.668327, -56.69416 ,
       -56.8258  , -58.064236, -57.938488, -56.90294 , -56.90026 ,
       -56.48227 , -58.99698 , -58.85235 , -58.181664, -55.98642 ,
       -57.59114 , -58.67091 , -59.410954, -58.078316, -58.395626,
       -56.53549 , -56.802666, -57.817825, -57.55971 , -57.068478,
       -55.886612, -55.62043 , -56.85259 , -55.615112, -57.423187,
       -58.65168 , -58.25182 , -58.85849 , -58.846207, -59.66834 ,
       -60.100693, -60.817085, -61.510292, -61.84868 , -62.075718,
       -62.074112, -62.60141 , -62.69142 , -60.156475, -58.66417 ,
       -58.836452, -58.94244 , -59.174664, -58.008244, -57.496716,
       -56.099037, -56.071007, -54.518993, -54.06901 , -54.937103,
       -54.929108, -54.014652, -54.42614 , -54.037262, -54.998

'id'

(13083,)

'XC511677.mp3'

# **Random Forest Model**

Saving evaluation results

In [15]:
def evaluate_model(rf, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = rf.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = rf.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [16]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [17]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)

training_features.shape

(42578, 244)

In [18]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)

validation_features.shape

(13083, 244)

Fit the model with training data

In [19]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [20]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['all_features'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['all_features'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2773828632576626


In [21]:
val_scores['all_features'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.25      0.43      0.32       793
           1       0.36      0.55      0.44      1294
           2       0.20      0.33      0.25       617
           3       0.26      0.31      0.29       984
           4       0.01      0.02      0.02       229
           5       0.00      0.00      0.00       303
           6       0.30      0.04      0.07       275
           7       0.39      0.39      0.39       698
           8       0.14      0.52      0.22       176
           9       0.13      0.31      0.18       227
          10       0.37      0.20      0.26       455
          11       0.36      0.01      0.02      1660
          12       0.32      0.29      0.30      1261
          13       0.39      0.10      0.15       502
          14       0.02      0.06      0.04       156
          15       0.24      0.30      0.26       798
          16       0.69      0.60      0.64   

### **With MFCCs only**

In [22]:
training_features = train_features['mfcc']

training_features.shape

(42578, 20)

In [23]:
validation_features = val_features['mfcc']

validation_features.shape

(13083, 20)

In [24]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [25]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.31758770924099977


In [26]:
val_scores['mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.28      0.66      0.40       793
           1       0.47      0.50      0.48      1294
           2       0.17      0.36      0.24       617
           3       0.59      0.65      0.62       984
           4       0.04      0.03      0.03       229
           5       0.00      0.00      0.00       303
           6       0.09      0.02      0.03       275
           7       0.33      0.53      0.41       698
           8       0.27      0.49      0.35       176
           9       0.17      0.35      0.23       227
          10       0.41      0.25      0.31       455
          11       0.63      0.02      0.04      1660
          12       0.35      0.34      0.35      1261
          13       0.58      0.23      0.33       502
          14       0.06      0.09      0.07       156
          15       0.17      0.35      0.23       798
          16       0.56      0.24      0.34   

### **With Mel-Spectrogram Only**

In [27]:
training_features = train_features['melspectrogram']

training_features.shape

(42578, 128)

In [28]:
validation_features = val_features['melspectrogram']

validation_features.shape

(13083, 128)

In [29]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [30]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 0.9999765136925173
Validation accuracy: 0.27019796682718034


In [31]:
val_scores['melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.24      0.46      0.32       793
           1       0.33      0.54      0.41      1294
           2       0.21      0.36      0.27       617
           3       0.32      0.29      0.30       984
           4       0.02      0.03      0.02       229
           5       0.00      0.00      0.00       303
           6       0.16      0.03      0.04       275
           7       0.30      0.28      0.29       698
           8       0.14      0.54      0.22       176
           9       0.19      0.28      0.23       227
          10       0.33      0.18      0.24       455
          11       0.17      0.01      0.02      1660
          12       0.25      0.19      0.22      1261
          13       0.22      0.07      0.10       502
          14       0.03      0.06      0.04       156
          15       0.27      0.36      0.31       798
          16       0.73      0.73      0.73   

## **With Chroma Only**

In [32]:
training_features = train_features['chroma']

training_features.shape

(42578, 12)

In [33]:
validation_features = val_features['chroma']

validation_features.shape

(13083, 12)

In [34]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [35]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma'] = rf.score(validation_features, val_y)

Training accuracy: 0.9996946780027244
Validation accuracy: 0.18940609951845908


In [36]:
val_scores['chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.12      0.46      0.18       793
           1       0.27      0.34      0.30      1294
           2       0.10      0.39      0.16       617
           3       0.53      0.42      0.47       984
           4       0.16      0.03      0.05       229
           5       0.41      0.10      0.16       303
           6       0.00      0.00      0.00       275
           7       0.16      0.24      0.19       698
           8       0.04      0.02      0.02       176
           9       0.16      0.10      0.12       227
          10       0.03      0.02      0.02       455
          11       0.14      0.00      0.01      1660
          12       0.17      0.06      0.09      1261
          13       0.15      0.04      0.06       502
          14       0.01      0.01      0.01       156
          15       0.28      0.49      0.36       798
          16       0.35      0.24      0.29   

## **With CQT Only**

In [37]:
training_features = train_features['cqt']

training_features.shape

(42578, 84)

In [38]:
validation_features = val_features['cqt']

validation_features.shape

(13083, 84)

In [39]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [40]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.1935335932125659


In [41]:
val_scores['cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.11      0.22      0.15       793
           1       0.34      0.45      0.39      1294
           2       0.00      0.01      0.00       617
           3       0.26      0.29      0.27       984
           4       0.01      0.01      0.01       229
           5       0.00      0.00      0.00       303
           6       0.07      0.03      0.04       275
           7       0.34      0.38      0.36       698
           8       0.15      0.47      0.22       176
           9       0.06      0.12      0.08       227
          10       0.15      0.08      0.10       455
          11       0.06      0.00      0.00      1660
          12       0.19      0.17      0.18      1261
          13       0.38      0.09      0.15       502
          14       0.02      0.06      0.03       156
          15       0.16      0.18      0.17       798
          16       0.45      0.49      0.47   

## **With MFCCs and Mel-Spectrogram**

In [42]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(42578, 148)

In [43]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(13083, 148)

In [44]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [45]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.26393029121761064


In [46]:
val_scores['mfcc_melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.26      0.49      0.34       793
           1       0.32      0.55      0.41      1294
           2       0.22      0.35      0.27       617
           3       0.37      0.36      0.37       984
           4       0.01      0.02      0.02       229
           5       0.00      0.00      0.00       303
           6       0.21      0.04      0.07       275
           7       0.33      0.33      0.33       698
           8       0.14      0.52      0.22       176
           9       0.11      0.29      0.16       227
          10       0.33      0.17      0.23       455
          11       0.30      0.01      0.02      1660
          12       0.30      0.23      0.26      1261
          13       0.40      0.13      0.20       502
          14       0.02      0.06      0.03       156
          15       0.26      0.35      0.30       798
          16       0.53      0.31      0.39   

## **With MFCCs and Chroma**

In [47]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(42578, 32)

In [48]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(13083, 32)

In [49]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [50]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3485439119468012


In [51]:
val_scores['mfcc_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.28      0.70      0.40       793
           1       0.46      0.52      0.49      1294
           2       0.22      0.46      0.30       617
           3       0.59      0.67      0.63       984
           4       0.08      0.04      0.05       229
           5       0.02      0.01      0.01       303
           6       0.29      0.04      0.08       275
           7       0.31      0.47      0.37       698
           8       0.26      0.49      0.34       176
           9       0.22      0.37      0.28       227
          10       0.45      0.25      0.32       455
          11       0.49      0.02      0.04      1660
          12       0.35      0.31      0.33      1261
          13       0.56      0.35      0.43       502
          14       0.03      0.04      0.04       156
          15       0.22      0.44      0.29       798
          16       0.73      0.49      0.59   

## **With MFCCs and CQT**

In [52]:
training_features = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)

training_features.shape

(42578, 104)

In [53]:
validation_features = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)

validation_features.shape

(13083, 104)

In [54]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [55]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.23511427042727204


In [56]:
val_scores['mfcc_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.18      0.37      0.24       793
           1       0.34      0.46      0.39      1294
           2       0.02      0.04      0.03       617
           3       0.26      0.33      0.29       984
           4       0.03      0.03      0.03       229
           5       0.00      0.00      0.00       303
           6       0.04      0.01      0.02       275
           7       0.36      0.45      0.40       698
           8       0.13      0.46      0.21       176
           9       0.20      0.26      0.23       227
          10       0.08      0.04      0.05       455
          11       0.28      0.01      0.01      1660
          12       0.36      0.35      0.35      1261
          13       0.28      0.05      0.09       502
          14       0.02      0.05      0.03       156
          15       0.17      0.19      0.18       798
          16       0.57      0.54      0.56   

## **With Chroma and CQT**

In [57]:
training_features = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)

training_features.shape

(42578, 96)

In [58]:
validation_features = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)

validation_features.shape

(13083, 96)

In [59]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [60]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2114958342887717


In [61]:
val_scores['chroma_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.12      0.24      0.16       793
           1       0.38      0.46      0.41      1294
           2       0.01      0.01      0.01       617
           3       0.22      0.30      0.25       984
           4       0.01      0.01      0.01       229
           5       0.00      0.00      0.00       303
           6       0.05      0.02      0.03       275
           7       0.31      0.36      0.33       698
           8       0.16      0.46      0.24       176
           9       0.10      0.15      0.12       227
          10       0.09      0.05      0.06       455
          11       0.13      0.00      0.01      1660
          12       0.28      0.27      0.27      1261
          13       0.37      0.08      0.13       502
          14       0.02      0.05      0.03       156
          15       0.21      0.25      0.23       798
          16       0.50      0.57      0.53   

## **With Mel-Spectrogram and Chroma**

In [62]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(42578, 140)

In [63]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(13083, 140)

In [64]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [65]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 0.9999765136925173
Validation accuracy: 0.2665290835435298


In [66]:
val_scores['melspectrogram_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.24      0.46      0.31       793
           1       0.32      0.53      0.40      1294
           2       0.21      0.35      0.26       617
           3       0.34      0.31      0.33       984
           4       0.01      0.02      0.01       229
           5       0.00      0.00      0.00       303
           6       0.20      0.03      0.06       275
           7       0.34      0.32      0.33       698
           8       0.14      0.52      0.21       176
           9       0.17      0.28      0.21       227
          10       0.27      0.18      0.22       455
          11       0.28      0.01      0.02      1660
          12       0.26      0.18      0.21      1261
          13       0.29      0.08      0.13       502
          14       0.03      0.06      0.04       156
          15       0.27      0.37      0.31       798
          16       0.65      0.58      0.61   

## **With Mel-Spectrogram and CQT**

In [67]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)

training_features.shape

(42578, 212)

In [68]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)

validation_features.shape

(13083, 212)

In [69]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [70]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2585798364289536


In [71]:
val_scores['melspectrogram_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.22      0.37      0.28       793
           1       0.34      0.53      0.42      1294
           2       0.19      0.32      0.24       617
           3       0.25      0.28      0.27       984
           4       0.02      0.03      0.02       229
           5       0.00      0.00      0.00       303
           6       0.15      0.03      0.04       275
           7       0.42      0.38      0.40       698
           8       0.13      0.51      0.21       176
           9       0.11      0.30      0.16       227
          10       0.41      0.22      0.28       455
          11       0.16      0.01      0.01      1660
          12       0.27      0.26      0.27      1261
          13       0.38      0.11      0.17       502
          14       0.03      0.06      0.04       156
          15       0.21      0.24      0.22       798
          16       0.68      0.60      0.63   

## **With Mel-Spectrogram, MFCCs, CQT**

In [72]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)

training_features.shape

(42578, 232)

In [73]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)

validation_features.shape

(13083, 232)

In [74]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [75]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.274096155316059


In [76]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.26      0.41      0.31       793
           1       0.33      0.54      0.41      1294
           2       0.18      0.32      0.23       617
           3       0.26      0.29      0.28       984
           4       0.02      0.02      0.02       229
           5       0.00      0.00      0.00       303
           6       0.12      0.02      0.03       275
           7       0.39      0.39      0.39       698
           8       0.14      0.52      0.22       176
           9       0.14      0.30      0.19       227
          10       0.37      0.19      0.25       455
          11       0.26      0.01      0.02      1660
          12       0.30      0.31      0.30      1261
          13       0.41      0.09      0.15       502
          14       0.02      0.06      0.03       156
          15       0.22      0.26      0.24       798
          16       0.71      0.65      0.68   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [77]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(42578, 160)

In [78]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(13083, 160)

In [79]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [80]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2903768248872583


In [81]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.27      0.52      0.35       793
           1       0.33      0.54      0.41      1294
           2       0.23      0.35      0.28       617
           3       0.36      0.36      0.36       984
           4       0.01      0.01      0.01       229
           5       0.00      0.00      0.00       303
           6       0.28      0.04      0.06       275
           7       0.30      0.30      0.30       698
           8       0.15      0.54      0.23       176
           9       0.16      0.29      0.20       227
          10       0.30      0.18      0.23       455
          11       0.48      0.01      0.02      1660
          12       0.33      0.24      0.28      1261
          13       0.50      0.16      0.24       502
          14       0.02      0.06      0.03       156
          15       0.29      0.39      0.33       798
          16       0.68      0.61      0.64   

## Review results from all models

In [82]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.277
3,chroma,1.0,0.189
8,chroma_cqt,1.0,0.211
4,cqt,1.0,0.194
2,melspectrogram,1.0,0.270
9,melspectrogram_chroma,1.0,0.267
12,melspectrogram_chroma_mfcc,1.0,0.290
10,melspectrogram_cqt,1.0,0.259
11,melspectrogram_cqt_mfcc,1.0,0.274
1,mfcc,1.0,0.318


In [83]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(2)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features      0.26       0.75
3                    chroma      0.16       0.67
8                chroma_cqt      0.19       0.72
4                       cqt      0.18       0.72
2            melspectrogram      0.24       0.76
9     melspectrogram_chroma      0.24       0.76
10       melspectrogram_cqt      0.24       0.75
11  melspectrogram_cqt_mfcc      0.27       0.77
1                      mfcc      0.30       0.80
6               mfcc_chroma      0.33       0.82
7                  mfcc_cqt      0.21       0.73
5       mfcc_melspectrogram      0.24       0.77
